# Import libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from nltk.corpus import stopwords

import re

from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.models import load_model

from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.preprocessing import text
from keras.optimizers import Adam
# outdated: from keras.preprocessing import sequence 
from keras.utils.data_utils import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences

import squarify
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

2023-09-12 19:31:06.226129: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Load Final GRU model

In [2]:
from tensorflow.keras.models import load_model

loaded_model = load_model('final_gru_model.h5')

2023-09-12 19:31:13.839598: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Function for cleaning + lemmatizing data

In [3]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

# Define a function to preprocess the tweets
def preprocess_tweet(tweet):
    # Remove URLs
    tweet = re.sub(r'http\S+', '', tweet)
    
    # Convert to lowercase
    tweet = tweet.lower()
    
    # Tokenize the tweet
    words = word_tokenize(tweet)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    # Join words back into a sentence
    cleaned_tweet = ' '.join(words)
    
    return cleaned_tweet

# Load Tweets

In [4]:
trump_tweets_df = pd.read_csv('trumptweets.csv', encoding='ISO-8859-1')
trump_tweets_df.head()

,Tweets
0,Thank you to all of the television viewers tha...
1,Can you imagine if I had the small crowds that...
2,NATO commander agrees members should pay up vi...
3,"Wow, NATO's top commander just announced that ..."
4,"The ""Rust Belt"" was created by politicians lik..."


In [5]:
print(type(trump_tweets_df))

<class 'pandas.core.frame.DataFrame'>


# Clean Tweets

In [6]:
# Preprocess Trump tweets using preprocess_tweet function
trump_tweets_df['cleaned_tweet'] = trump_tweets_df['Tweets'].apply(preprocess_tweet)

# Preprocess tweets and predict MBTI

In [7]:
import joblib
labelencoder = joblib.load('labelencoder.pkl')
# Define the maximum number of words to keep
max_words = 2500

# Create a tokenizer
tokenizer = Tokenizer(num_words=max_words, filters="")

# Tokenize and pad sequences
max_len = 40  # Adjust to match the model's input length
X_trump = tokenizer.texts_to_sequences(trump_tweets_df['cleaned_tweet'])
X_trump = pad_sequences(X_trump, padding='post', maxlen=max_len)

# Make a single prediction for all the tweets using the pre-trained model
predictions = loaded_model.predict(X_trump)

# Decode the prediction back to an MBTI type
predicted_mbti_enc = np.argmax(predictions, axis=1)  # Get the index of the highest probability
predicted_mbti = labelencoder.inverse_transform(predicted_mbti_enc)  # Inverse transform to get the MBTI label

# Print the predicted MBTI type
print("Predicted MBTI Type for Trump's Tweets:", predicted_mbti[0])

934/934 [==============================] - 11s 11ms/step
Predicted MBTI Type for Trump's Tweets: INFP
